* Updated to GDS 2.0 version
* Link to original blog post: https://towardsdatascience.com/exploring-the-nft-transaction-with-neo4j-cba80ead7e0b

In [ ]:
from neo4j import GraphDatabase
import pandas as pd

host = 'bolt://localhost:7687'
user = 'neo4j'
password = 'letmein'
driver = GraphDatabase.driver(host,auth=(user, password))

def neo4j_query(query, params=None):
    with driver.session() as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [ ]:
neo4j_query("""
CALL apoc.warmup.run
""")

,pageSize,totalTime,transactionWasTerminated,nodesPerPage,nodesTotal,nodePages,nodesTime,relsPerPage,relsTotal,relPages,...,stringPropRecordsTotal,stringPropPages,stringPropsTime,arrayPropsPerPage,arrayPropRecordsTotal,arrayPropPages,arrayPropsTime,indexesLoaded,indexPages,indexTime
0,8192,4058,False,0,11286844,24576,108,0,22891323,106496,...,0,0,0,0,0,0,0,False,0,0


# Exploring the NFT transaction with Neo4j
## Exploratory graph analysis of 6 million NFT transactions
A couple of months ago, an article in Nature was published that maps 6.1 million transactions between June 2017 and April 2021 obtained primarily from Ethereum and WAX blockchains. It is an exciting article that analyzes the statistical properties of the market as well as some network characteristics. The authors also made the underlying NFT transaction dataset available under the CC-BY 4.0 license.
Transactions can be described as links between the traders, so what better way to represent the data than a graph. First, Michael Hunger and I imported the dataset into Neo4j, a native graph database. Then I performed a simple exploratory graph analysis to better understand the underlying dataset, which I will demonstrate in this blog post.
Setting up the environment
First, you need to download and install the Neo4j Desktop application. The dataset is available as a database dump, which you can download by clicking this link. You need to copy the file to the Neo4j Desktop files folder and then select Create new DBMS from dump.

It is advisable to choose the latest version of the Neo4j database available. Next, you will also need to install the APOC and GDS libraries.

As this is a slightly larger dataset, increasing the heap and page cache memory allocation is also recommended. I chose 4GB of memory for heap allocation and 2GB for the page cache.

Congratulations, you have successfully prepared the Neo4j environment.
## Graph schema model

The dataset revolves around transactions between traders. Each transaction has a date and the price in crypto and USD. As this is a dataset of NFT transactions, each transaction describes which NFT was sold. Interestingly, an image URL is available for most NFTs so that you can inspect the image sold on the internet. Some of the NFTs have up to four available image URLs for redundancy. We also know if an NFT is a part of a collection and which category it belongs to.
## Exploratory graph analysis
Now we can begin with the exploratory graph analysis. As always, I have prepared a Jupyter notebook that contains all the queries in this post.
First, we will evaluate the number of nodes and relationships with the `apoc.meta.stats` procedure.

In [ ]:
neo4j_query("""
CALL apoc.meta.stats
YIELD nodeCount, relCount, labels
""")

,nodeCount,relCount,labels
0,11286844,22891323,"{'Category': 6, 'Transaction': 6071027, 'Trade..."


There are around 11 million nodes and 23 million relationships in the graph. The dataset contains 6 million transactions for around 4.6 million NFTs. The NFTs are part of 4600 collections and 6 categories.
Now we will dig deeper and start examining the count of transactions and total volume and the average price of NFTs by year.

In [ ]:
neo4j_query("""
MATCH (t:Transaction)
RETURN t.Datetime_updated.year AS year, 
       count(*) AS transactions, 
       sum(t.Price_USD) AS totalVolume, 
       avg(t.Price_USD) AS averagePrice
ORDER BY year 
""")

,year,transactions,totalVolume,averagePrice
0,2017,253100,1.829279e+07,72.274933
1,2018,449373,1.667865e+07,37.118441
2,2019,746489,1.867730e+07,25.086969
3,2020,1245954,7.762350e+07,62.551672
4,2021,3376111,7.559576e+08,223.996997


In 2018, there were only 450 thousand transactions, whereas, in 2021, there were almost three and a half million transactions till April. Over the years, the average price and the total volume have been rising to nearly 750 million USD in the first four months of 2021. There was definitely a boom in the NFT world.
Next, we will evaluate how many transactions have been priced under or above one dollar.

In [ ]:
neo4j_query("""
MATCH (t:Transaction)
WHERE exists(t.Price_USD)
RETURN CASE WHEN t.Price_USD > 1 THEN true ELSE false END AS moreThanDollar, count(*) AS count
""")

,moreThanDollar,count
0,False,2730965
1,True,3331779


Around 55% percent of the transactions have been priced over one dollar. That leaves about 2.7 million transactions that were under a dollar.
It seems that the high-value transactions are not that frequent, so when some occur, it makes the news. Let's examine the distribution of transactions over one dollar with the `apoc.agg.statistics` procedure.

In [ ]:
neo4j_query("""
MATCH (t:Transaction)
WHERE exists(t.Price_USD) AND t.Price_USD > 1
RETURN apoc.agg.statistics(t.Price_USD) AS result
""")

,result
0,"{'total': 3331779, 'min': 1.0000050750633935, ..."


The average price of transactions over one dollar is 266 USD. On the other hand, 99 percent of transactions were under 3349 USD. That means that only a tiny percent of transactions were  very expensive, with the highest value of transaction being 7.5 million USD.
We can examine which NFTs were sold at the highest price with the following Cypher statement.

In [ ]:
neo4j_query("""
MATCH (n:NFT)<-[:FOR_NFT]-(t:Transaction)
WHERE exists(t.Price_USD)
WITH n, t.Price_USD as price
ORDER BY price DESC LIMIT 5
RETURN n.ID_token as token_id, n.Image_url_1 as image_url, price
""")

,token_id,image_url,price
0,"('Cryptopunks', '7804')",https://lh3.googleusercontent.com/KPQPI9GRwIMP...,7.501893e+06
1,"('Cryptopunks', '3100')",https://lh3.googleusercontent.com/q9gazeJi0cEH...,7.501893e+06
2,"('Somnium-space', '0')",https://lh3.googleusercontent.com/MWyOpZRNd0Ln...,2.684347e+06
3,"('Cryptopunks', '3011')",https://lh3.googleusercontent.com/tblRChM9Ych3...,1.723131e+06
4,"('Superrare', '14')",https://lh3.googleusercontent.com/Fzc7-PJ-e1YC...,1.643615e+06


As a data analyst, I am very suspicious that two NFTs were valued at identical prices, and on top of that, they are both for 7.5 million USD. As we have the image URL available, you can inspect which images were valued at over a million USD. It looks like Cryptopunks collectibles are highly valued.
We can dig deeper and inspect which accounts were involved in the 7.5 million USD transactions.

In [ ]:
neo4j_query("""
MATCH (n:NFT)<-[:FOR_NFT]-(t:Transaction),
      (t)<-[:BOUGHT]-(buyer),
      (t)<-[:SOLD]-(seller)
WHERE t.Price_USD = 7501893.0
RETURN buyer.address AS buyer, 
       seller.address AS seller, 
       t.Datetime_updated_seconds AS date, 
       t.Price_USD AS price
""")

,buyer,seller,date,price
0,0x0000000000000000000000000000000000000000,0x6611fe71c233e4e7510b2795c242c9a57790b376,2021-03-11T20:57:34.000000000+00:00,7501893.0
1,0xf4b4a58974524e183c275f3c6ea895bc2368e738,0x03911fecabd6b4809c88e2e6eb856ec932b2ee3e,2021-03-11T00:49:11.000000000+00:00,7501893.0


What a strange coincidence that both transactions were executed on the same day, only around 19 hours in between. The buyers and sellers seem unrelated, although the address that contains all zeroes caught my eye. After some investigating, I've learned that when someone mints an NFT on the OpenSea platform, the null address always shows up as a creator. On the other hand, you could also send an NFT to the null address to effectively delete the NFT. At least that's why some of the posts on the internet claim. After I put on my Sherlock Holmes hat, I found the NullAddress profile on OpenSea, where you can buy NFTs from them. This left me even more confused as I would assume that selling an NFT to the null address would effectively burn the NFT token, but who would send a 7.5 million USD worth NFT to burn it. Perhaps I just misunderstood it all, as this is my first analysis of the NFT ecosystem, and I have no prior knowledge of it. In any case, it all seems a bit suspicious. The two transactions have an identical price and were executed on the same day. I've checked if the three addresses have any transactions between each other, but the dataset contains no such transactions.


Moving on, we will evaluate the NFT collections and their average cost.

In [ ]:
neo4j_query("""
MATCH (collection)<-[:IN_COLLECTION]-(n:NFT)<-[:FOR_NFT]-(t:Transaction)
WHERE exists (t.Price_USD)
RETURN collection.Collection AS collection, 
       avg(t.Price_USD) AS averagePrice, 
       count(distinct n) AS numberOfNfts
ORDER BY averagePrice DESC
LIMIT 5""")

,collection,averagePrice,numberOfNfts
0,Saturdaynightlive,360856.408928,1
1,Appresidential,181832.879943,1
2,Trippderrickbarnesxflipkick,118838.578811,1
3,Fairumnft,99849.750000,1
4,Chainsaw,96546.024285,12


Most of the highest-value collections have only a single NFT in our dataset. Interestingly, the Cryptopunks don't even make it to the top five, even though some million USD transactions are around them.
Next, we will explore which traders executed the most transactions.

In [ ]:
neo4j_query("""
MATCH (t:Trader)
RETURN coalesce(t.username, t.address) AS username,
       size((t)-[:BOUGHT]->()) AS bought,
       size((t)-[:SOLD]->()) AS sold
ORDER BY bought + sold desc LIMIT 5;
""")

,username,bought,sold
0,0x76481caa104b5f6bccb540dae4cefaf1c398ebea,130231,192586
1,0x327305a797d92a39cee1a225d7e2a1cc42b1a8fa,0,149142
2,0x4FabDA,28,49338
3,0xfc624f8f58db41bdb95aedee1de3c1cf047105f1,1976,43571
4,StrongHands,43384,437


The top five traders have bought or sold more than 40 thousand NFTs. Since all the data is available on the internet as well, we can look at the top profile. It seems they haven't made a single transactions since April 2020. On the other hand, StrongHands profile on OpenSea has 17 thousand NFTs one could purchase.
We have looked at traders with the highest transactions count, and now we will look at traders with the highest volume in USD.

In [ ]:
neo4j_query("""
MATCH (t:Trader)
OPTIONAL MATCH (t)-[:BOUGHT]->(bt)
WITH t, sum(bt.Price_USD) AS boughtVolume
OPTIONAL MATCH (t)-[:SOLD]->(st)
WITH t, boughtVolume, sum(st.Price_USD) AS soldVolume
RETURN t.username AS username, t.address AS address,boughtVolume, soldVolume
ORDER BY boughtVolume + soldVolume
DESC LIMIT 5
""")

,username,address,boughtVolume,soldVolume
0,NullAddress,0x0000000000000000000000000000000000000000,5.506430e+07,3.142260e+03
1,None,0x327305a797d92a39cee1a225d7e2a1cc42b1a8fa,0.000000e+00,2.660329e+07
2,SethS,0x6611fe71c233e4e7510b2795c242c9a57790b376,8.304005e+05,1.314407e+07
3,Pranksy,0xd387a6e4e84a6c86bd90c158c6028a58cc8ac459,3.931823e+06,9.985689e+06
4,SomniumSpace,0xb98cdacd006b9d47c37ca63cc86f916ee23fc550,1.166480e+03,1.310994e+07


The highest volume has the NullAddress account. I have no idea why it has over 50 million USD in buyings. Even before, we had an example where someone sent an NFT to NullAddress for 4200 ETH. Maybe you could pick an arbitrary number of the transaction amount when you burn the NFT. Perhaps some experts can help me out with this.
On the other hand, the second address is only interested in selling, which is also peculiar. I've inspected the highest transaction of the address in the dataset, and it seems this is a wallet for the Sorare project, which looks like fantasy football with NFT tokens. My guess is that they mint the tokens themselves and only sell them to their platform users.
Next, we will explore which users have resold an NFT with the highest profit.

In [ ]:
neo4j_query("""
MATCH (t:Trader)-[:SOLD]->(st:Transaction)-[:FOR_NFT]->(nft)
WHERE st.Price_USD > 100000
MATCH (t)-[:BOUGHT]->(bt:Transaction)-[:FOR_NFT]->(nft)
WHERE st.Datetime_updated_seconds > bt.Datetime_updated_seconds
RETURN t.username as trader, nft.Image_url_1 as nft, nft.ID_token AS tokenID, 
       st.Datetime_updated_seconds AS soldTime, 
       st.Price_USD AS soldAmount, 
       bt.Datetime_updated_seconds as boughtTime, 
       bt.Price_USD AS boughtAmount,
       st.Price_USD - bt.Price_USD AS difference
ORDER BY difference DESC LIMIT 5""")

,trader,nft,tokenID,soldTime,soldAmount,boughtTime,boughtAmount,difference
0,None,https://lh3.googleusercontent.com/tblRChM9Ych3...,"('Cryptopunks', '3011')",2021-04-27T15:44:50.000000000+00:00,1723131.135,2021-03-12T01:00:13.000000000+00:00,712808.00,1010323.135
1,PixelVault_,https://lh3.googleusercontent.com/cBxKNeyUzvwl...,"('Superrare', '12')",2021-03-31T15:39:49.000000000+00:00,883388.375,2021-03-18T19:09:05.000000000+00:00,99253.00,784135.375
2,None,https://lh3.googleusercontent.com/l6EtSH2XoyOi...,"('Cryptopunks', '2306')",2021-04-18T11:04:44.000000000+00:00,994856.440,2021-02-11T04:01:32.000000000+00:00,220262.50,774593.940
3,TokenAngels,https://lh3.googleusercontent.com/tblRChM9Ych3...,"('Cryptopunks', '3011')",2021-03-12T01:00:13.000000000+00:00,712808.000,2020-10-12T18:11:59.000000000+00:00,15981.84,696826.160
4,Pranksy,https://lh3.googleusercontent.com/1Tc5JdWMknIk...,"('Cryptopunks', '2140')",2021-03-02T01:02:54.000000000+00:00,1147815.000,2021-02-22T10:44:16.000000000+00:00,676912.00,470903.000


It seems that Cryptopunk were (are?) the most lucrative tokens. For example, a user bought the Cyptopunk 3011 NFT on the 12th of March 2021 at 700 thousand USD and then resold it a good month later for 1.7 million USD. That is a good deal for a pixel image of a dude smoking a pipe. Now, you might say why would someone sell an NFT for 700 thousand when they could get a million more, but here is the catch. That person bought it for 16 thousand USD in October 2020 and sold it in March 2021 for 700 thousand. So effectively, the price of the NFT went 100x from October 2020 to March 2021.
## Graph algorithms
Lastly, we will run few graph algorithms on the dataset. We will construct an inferred directed weighted network of transactions between traders. The direction of the relationship will indicate the flow of the money, while the weight will represent the volume. We will use the Cypher Projection to project a virtual network that is a transformed version of the underlying stored graph.

In [ ]:
neo4j_query("""
CALL gds.graph.project.cypher("nft", 
"MATCH (t:Trader) WHERE NOT t.username = 'NullAddress' RETURN id(t) as id",
"MATCH (t1:Trader)-[:BOUGHT]->(t)<-[:SOLD]-(t2:Trader)
 RETURN id(t1) AS source, id(t2) as target, apoc.coll.max([sum(coalesce(t.Price_USD,0.1)),1]) as weight", {validateRelationships:false})
""")

,nodeQuery,relationshipQuery,graphName,nodeCount,relationshipCount,createMillis
0,MATCH (t:Trader) WHERE NOT t.username = 'NullA...,MATCH (t1:Trader)-[:BOUGHT]->(t)<-[:SOLD]-(t2:...,nft,70805,338639,77408


The first algoritm we will execute is the Weakly Connected Component algorithm. It is used to find disconnected parts or islands within the network.

In [ ]:
neo4j_query("""
CALL gds.wcc.stats("nft")
YIELD componentCount, componentDistribution
""")

,componentCount,componentDistribution
0,5287,"{'p99': 3, 'min': 1, 'max': 64774, 'mean': 13...."


The NFT transactions network is quite disconnected. There are 5287 separate components, and the largest one contains only around 12% of traders. That means that there are several small components where traders only conducted business between a small group of users. What's even stranger is that more than 75% of components consist of a single user. That means that bought the sender, as well as the recipient, is the same user.
Let's examine a sample trader with self-loops.

In [ ]:
neo4j_query("""
MATCH p=(t:Trader)-[:BOUGHT]->()<-[:SOLD]-(t)
WHERE t.username = "grake"
RETURN [node in nodes(p) | node.Transaction_hash] AS result
LIMIT 10
""")

,result
0,"[None, 0xa5437ea384eebabfef3347606055cfbb9ff44..."
1,"[None, 0xf325fe3959979471bf5a14f74fc658663424e..."
2,"[None, 0xdf321ceee408d6a1fd7a44a624da393e373d6..."
3,"[None, 0x735d66b71a437da8ef2c3220b83d42ec5c7d5..."


At first, I thought this was a mistake in the underlying dataset. However, I inspected a single transaction on the OpenSea platform, and it looks like a user can sell an NFT to themselves. However, I think that there are some mechanics of the platform, which leaves the logs confusing.

Lastly. we will execute the ArticleRank algorithm, which is a variation of the PageRank algorithm to find the network influencers.

In [ ]:
neo4j_query("""
CALL gds.articleRank.stream("nft")
YIELD nodeId, score
WITH gds.util.asNode(nodeId) AS node, score
ORDER BY score DESC
LIMIT 5
RETURN node.username as username,
      score,
      size((node)-[:BOUGHT]->()) as boughtCount,
      size((node)-[:SOLD]->()) as soldCount,
      apoc.coll.sum([(node)-[:BOUGHT]->(t) | coalesce(t.Price_USD,0) ]) as boughtVolume,
      apoc.coll.sum([(node)-[:SOLD]->(t) | coalesce(t.Price_USD,0) ]) as soldVolume
""")

,username,score,boughtCount,soldCount,boughtVolume,soldVolume
0,YellowHeartFactories,81.984069,3,5423,5.176749e+03,7.096732e+05
1,Pranksy,35.339142,3044,20198,3.931823e+06,9.985689e+06
2,0xfc624f8f58db41bdb95aedee1de3c1cf047105f1,27.586534,1976,43571,1.299069e+05,3.551704e+05
3,0x4FabDA,23.406713,28,49338,6.820018e+04,5.425523e+05
4,ethernitychain,17.147044,0,1055,NaN,6.286660e+05


In [ ]:
neo4j_query("""
CALL gds.graph.drop('nft')
""")

,graphName,database,memoryUsage,sizeInBytes,nodeProjection,relationshipProjection,nodeQuery,relationshipQuery,nodeCount,relationshipCount,nodeFilter,relationshipFilter,density,creationTime,modificationTime,schema
0,nft,neo4j,,-1,None,None,MATCH (t:Trader) WHERE NOT t.username = 'NullA...,MATCH (t1:Trader)-[:BOUGHT]->(t)<-[:SOLD]-(t2:...,70805,338639,None,None,0.000068,2022-01-08T20:39:24.267423000+01:00,2022-01-08T20:40:41.680117000+01:00,{'relationships': {'__ALL__': {'weight': 'Floa...
